In [3]:
import geopandas as gpd
from tqdm import tqdm
import time

"""Add stops to routes"""
# Filepath
bus_routes_fp = "../RTA_Bus_Routes-shp/RTA_Bus_Routes.shp"
bus_stops_fp = "../RTA_Bus_Stops-shp/RTA_Bus_Stops.shp"

# Read the data
bus_routes_df = gpd.read_file(bus_routes_fp)
bus_stops_df = gpd.read_file(bus_stops_fp)

# Pick specific columns
routes = bus_routes_df[['OBJECTID','geometry','route_id', 'route_shor', 'route_long']]
stops = bus_stops_df[['OBJECTID','geometry','stop_id']]

DriverError: ../RTA_Bus_Routes-shp/RTA_Bus_Routes.shp: No such file or directory

In [2]:
# map bus stops to routes
now = time.time()
mapped_routes = [-1] * len(stops)
current_route_id = ''

def distance_mapper(row): 
    stop = row['geometry']
    index = row.name
    global current_route_id
    
    # TODO check threshold
    if stop < 100:
        # print(current_route_id, index, stop)
        mapped_routes[index] = current_route_id
        
def distance_matrix(row):
    line = row.geometry
    global current_route_id
    current_route_id = row.route_id

    stops_distances = stops.apply(lambda stop: line.distance(stop.geometry), axis=1).to_frame('geometry')
    
    stops_distances.apply(distance_mapper, axis=1)
    
routes.apply(distance_matrix, axis=1)

stops['route_id'] = mapped_routes
print(f"Finished in {time.time()-now}s")
stops.to_csv('data/bus_stop_route_mapping.csv')

Finished in 928.358372926712s


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
